In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-19 07:31:51--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.7’

minsearch.py.7      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-19 07:31:51 (14.8 MB/s) - ‘minsearch.py.7’ saved [3832/3832]



In [2]:
%load_ext autoreload
%autoreload 2

In [22]:
import json
import requests
import minsearch
import os
from groq import Groq

def search(query):
    boost_dict = {"question": 3}

    results = index.search(query, filter_dict, boost_dict, num_results=10)

    return results

def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. Use only the facts from the CONTEXT from the FAQ database.
    If the CONTEXT doesn't contain the answer, type NONE.
    
    QUESTION: {question}
    Context: 
    {context}
    """.strip()

    context = ''

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",
        )

    return response.choices[0].message.content



docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

filter_dict = {"course": "data-engineering-zoomcamp"}

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)




def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    return llm(prompt)

query = "How cn I run kafka?"
rag("The course has already started. Can i still enroll?")
#rag("How can I run kafka?")

"Yes, you can still enroll in the course after it has already started. The context mentions that even if you don't register, you're still eligible to submit the homeworks."